In [1]:
import pandas as pd

In [2]:
schedule = pd.read_csv('../INPUT/data_fam_schedule.csv')

In [4]:
schedule

,flight,deptime,depoff,arrtime,arroff,origin,destination,Depart_time,Arrive_time,Flight_time
0,AA0001,755,-400,1038,-700,EDB,TFB,1155,1738,583
1,AA0002,1405,-700,2159,-400,TFB,EDB,2105,2559,454
2,AA0003,1740,-400,2025,-700,EDB,TFB,2140,2725,585
3,AA0004,840,-700,1640,-400,TFB,EDB,1540,2040,500
4,AA0005,910,-400,1202,-700,EDB,MBY,1310,1902,592
...,...,...,...,...,...,...,...,...,...,...
810,BA2645,1705,-700,1850,-700,QEY,TNG,2405,2550,145
811,BA2670,1335,-700,1435,-700,TFB,ZMX,2035,2135,100
812,BA2700,2016,-700,2230,-700,NGS,MBY,2716,2930,214
813,AA2701,645,-800,1243,-700,BOD,MBT,1445,1943,498


In [5]:
origin = schedule['origin'].tolist()
destination = schedule['destination'].tolist()

In [14]:
O_D = {}
for i in range(len(origin)):
    pair = (origin[i] , destination[i])
    if pair in O_D:
        O_D[pair] += 1
    else:
        O_D[pair] = 1

In [15]:
O_D

{('EDB', 'TFB'): 2,
 ('TFB', 'EDB'): 2,
 ('EDB', 'MBY'): 1,
 ('MBY', 'EDB'): 1,
 ('FXS', 'TFB'): 2,
 ('TFB', 'FXS'): 2,
 ('TFB', 'CPT'): 2,
 ('CPT', 'TFB'): 2,
 ('TFB', 'NJB'): 1,
 ('NJB', 'TFB'): 1,
 ('TFB', 'NDP'): 1,
 ('NDP', 'TFB'): 1,
 ('TFB', 'PSE'): 3,
 ('PSE', 'TFB'): 3,
 ('QEY', 'CPT'): 1,
 ('CPT', 'QEY'): 1,
 ('TFB', 'NTQ'): 2,
 ('NTQ', 'TFB'): 2,
 ('BER', 'BOD'): 2,
 ('BOD', 'CFU'): 3,
 ('CFU', 'BOD'): 3,
 ('BOD', 'BER'): 2,
 ('GBJ', 'CSX'): 1,
 ('BOD', 'GBJ'): 8,
 ('GBJ', 'BOD'): 7,
 ('CSX', 'GBJ'): 1,
 ('BOD', 'TDD'): 1,
 ('TDD', 'BOD'): 1,
 ('CSX', 'BOD'): 1,
 ('GBJ', 'TDD'): 1,
 ('TDD', 'CSX'): 1,
 ('KOV', 'TFB'): 4,
 ('TFB', 'KOV'): 4,
 ('LUO', 'TFB'): 3,
 ('KOV', 'LUO'): 1,
 ('KOV', 'ZBL'): 1,
 ('ZBL', 'DEW'): 1,
 ('DEW', 'BOD'): 1,
 ('BOD', 'KOV'): 3,
 ('KOV', 'TJU'): 2,
 ('TJU', 'LUO'): 1,
 ('KOV', 'QTH'): 1,
 ('QTH', 'XSH'): 1,
 ('XSH', 'LUO'): 1,
 ('KOV', 'BOD'): 3,
 ('LUO', 'XSH'): 1,
 ('QTH', 'KOV'): 1,
 ('TFB', 'LUO'): 3,
 ('XSH', 'QTH'): 1,
 ('BOD', 'DEW'): 1,


In [16]:
len(O_D)

297

In [31]:
pairs = { }
single = { }
checked = [ ]
imbalance = { }

In [32]:
for pair in O_D:
    reverse = (pair[1], pair[0])
    
    if pair in checked:
        continue
        
    checked.append(reverse)
    
    if reverse in O_D:
        pairs[pair] = f'forward: {O_D[pair]}, reverse: {O_D[reverse]}'
        if O_D[pair] != O_D[reverse]:
            imbalance[pair] = f'forward: {O_D[pair]}, reverse: {O_D[reverse]}'
    else:
        single[pair] = O_D[pair]
        

In [28]:
pairs

{('EDB', 'TFB'): 'forward: 2, reverse: 2',
 ('EDB', 'MBY'): 'forward: 1, reverse: 1',
 ('FXS', 'TFB'): 'forward: 2, reverse: 2',
 ('TFB', 'CPT'): 'forward: 2, reverse: 2',
 ('TFB', 'NJB'): 'forward: 1, reverse: 1',
 ('TFB', 'NDP'): 'forward: 1, reverse: 1',
 ('TFB', 'PSE'): 'forward: 3, reverse: 3',
 ('QEY', 'CPT'): 'forward: 1, reverse: 1',
 ('TFB', 'NTQ'): 'forward: 2, reverse: 2',
 ('BER', 'BOD'): 'forward: 2, reverse: 2',
 ('BOD', 'CFU'): 'forward: 3, reverse: 3',
 ('GBJ', 'CSX'): 'forward: 1, reverse: 1',
 ('BOD', 'GBJ'): 'forward: 8, reverse: 7',
 ('BOD', 'TDD'): 'forward: 1, reverse: 1',
 ('KOV', 'TFB'): 'forward: 4, reverse: 4',
 ('LUO', 'TFB'): 'forward: 3, reverse: 3',
 ('KOV', 'LUO'): 'forward: 1, reverse: 1',
 ('KOV', 'ZBL'): 'forward: 1, reverse: 1',
 ('ZBL', 'DEW'): 'forward: 1, reverse: 1',
 ('DEW', 'BOD'): 'forward: 1, reverse: 1',
 ('BOD', 'KOV'): 'forward: 3, reverse: 3',
 ('KOV', 'TJU'): 'forward: 2, reverse: 2',
 ('TJU', 'LUO'): 'forward: 1, reverse: 1',
 ('KOV', 'Q

In [29]:
single

{('CSX', 'BOD'): 1,
 ('GBJ', 'TDD'): 1,
 ('TDD', 'CSX'): 1,
 ('IMO', 'TFB'): 1,
 ('BOD', 'MBT'): 1}

In [33]:
imbalance

{('BOD', 'GBJ'): 'forward: 8, reverse: 7',
 ('BOD', 'TFB'): 'forward: 16, reverse: 17',
 ('BOD', 'PUA'): 'forward: 2, reverse: 1',
 ('PNF', 'BOD'): 'forward: 2, reverse: 1',
 ('PUA', 'PNF'): 'forward: 2, reverse: 1',
 ('QEY', 'MBT'): 'forward: 3, reverse: 4',
 ('QEY', 'PHH'): 'forward: 2, reverse: 1',
 ('PHH', 'TFB'): 'forward: 2, reverse: 1',
 ('IMO', 'HUG'): 'forward: 1, reverse: 2',
 ('HUG', 'TFB'): 'forward: 1, reverse: 2'}